In [91]:
import networkx as nx
import pandas as pd

## Reading reduced doc with edges

In [6]:
columns = ['source', 'destination']
reduced = pd.read_csv('wiki-topcats-reduced.txt', names=columns, sep='\t')

In [7]:
reduced[0:5]

,source,destination
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


## Reading pages names (NOT DONE YET)

In [8]:
columns = ['id', 'page_name']
pages_name = pd.read_csv('wiki-topcats-page-names.txt', names=columns, index_col=0)

In [9]:
pages_name[0:5]

,page_name
id,
0 Chiasmal syndrome,NaN
1 Kleroterion,NaN
2 Pinakion,NaN
3 LyndonHochschildSerre spectral sequence,NaN
4 Zariski's main theorem,NaN


## Reading categories 

In [33]:
categories = {}
with open('wiki-topcats-categories.txt') as file:
    for line in file:
        row_cat = line.rstrip().partition(':')[2].partition('; ')[0]
        row_artic = line.rstrip().partition(':')[2].partition('; ')[2].split()
        if len(row_artic) > 3500:
            categories[row_cat] = row_artic

In [34]:
categories.keys()

dict_keys(['English_footballers', 'The_Football_League_players', 'Association_football_forwards', 'Association_football_goalkeepers', 'Association_football_midfielders', 'Association_football_defenders', 'Living_people', 'Year_of_birth_unknown', 'Harvard_University_alumni', 'Major_League_Baseball_pitchers', 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 'Indian_films', 'Year_of_death_missing', 'English_cricketers', 'Year_of_birth_missing_(living_people)', 'Rivers_of_Romania', 'Main_Belt_asteroids', 'Asteroids_named_for_people', 'English-language_albums', 'English_television_actors', 'British_films', 'English-language_films', 'American_films', 'Fellows_of_the_Royal_Society', 'People_from_New_York_City', 'American_Jews', 'American_television_actors', 'American_film_actors', 'Debut_albums', 'Black-and-white_films', 'Year_of_birth_missing', 'Place_of_birth_missing_(living_people)', 'Article_Feedback_Pilot', 'American_military_personnel_of_World_War_II', 'Windows_gam

In [90]:
len(categories.get('English_footballers'))

9237

In [42]:
G_Cat=nx.Graph(categories)

# RQ1 Making the graph

In [86]:
G_Art=nx.Graph()
# Insert Nodes
G_Art.add_nodes_from(reduced['source'])
G_Art.add_nodes_from(reduced['destination'])

In [87]:
G_Art.number_of_nodes()

461193

In [57]:
G_Art.has_node(52)

True

In [68]:
len(reduced)

2645247

In [88]:
# Insert Edges
for index, row in reduced.iterrows():
    G_Art.add_edge(row['source'], row['destination'])

In [85]:
G_Art.number_of_edges()

9515

In [94]:
print(nx.info(G_Art))

Name: 
Type: Graph
Number of nodes: 461193
Number of edges: 2174451
Average degree:   9.4297
